In [4]:
"""Code use to scrap the data of the injury in NFL from 2000 to 2022 with the website prosportstransactions"""
"""Created by Antoine MICHEL"""

#import the library
import requests
import re
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

#Create a function

def replace_all(text, dic):
    '''
    This function will replace characters in text given a dictionary of characters to seach for and replace
    '''
    rc = re.compile('|'.join(map(re.escape, dic)))

    def translate(match):
        return dic[match.group(0)]
    return rc.sub(translate, text)

char_replace = {' • ': ''}

list_of_rows = []

#Get all links
#look the number of pages on the website
for i in range (0,6325,25):
    url = "https://www.prosportstransactions.com/football/Search/SearchResults.php?Player=&Team=&BeginDate=2015-01-01&EndDate=2022-07-22&InjuriesChkBx=yes&submit=Search&start={}".format(i)
    req = requests.get(url)
    html = req.content
    soup = BeautifulSoup(html, "lxml")

    table = soup.find('table', attrs={'class': 'datatable center'})

    for row in table.findAll ('tr', attrs={'align':'left'}):
        list_of_cells = []
    
        for cell in row.findAll ('td'):
            text = replace_all(cell.text,char_replace)
            text = text.strip()
            list_of_cells.append(text)
        list_of_rows.append(list_of_cells)



In [5]:
list_of_rows

[['2015-01-02', 'Bengals', '', 'James Wright (c)', 'knee injury (out)'],
 ['2015-01-02', 'Cardinals', '', 'Drew Stanton', 'knee injury (out)'],
 ['2015-01-02', 'Colts', '', 'Hugh Thornton', 'shoulder injury (out)'],
 ['2015-01-02', 'Lions', '', 'Larry Warford', 'knee injury (out)'],
 ['2015-01-02', 'Ravens', '', 'Eugene Monroe', 'ankle injury (out)'],
 ['2015-01-02',
  'Ravens',
  '',
  'Tim Jernigan / Timmy Jernigan',
  'foot injury (out)'],
 ['2015-01-02', 'Steelers', '', "Le'Veon Bell", 'knee injury (out)'],
 ['2015-01-09', 'Broncos', '', 'Quanterus Smith', 'knee injury (out)'],
 ['2015-01-09', 'Panthers', '', 'Star Lotulelei', 'foot injury (out)'],
 ['2015-01-09', 'Seahawks', '', 'Jeron Johnson', 'elbow injury (out)'],
 ['2015-01-16', 'Patriots', '', 'Bryan Stork', 'knee injury (out)'],
 ['2015-01-23', 'Patriots', '', 'Bryan Stork', 'knee injury (out)'],
 ['2015-05-05',
  'Panthers',
  '',
  'Bruce DeHaven',
  'took leave of absence as assistant special teams coach for health reaso

In [6]:
injuries_df_footus = pd.DataFrame(list_of_rows, columns = ['Date', 'team','to_delete','players','localisation'])

In [7]:
df = injuries_df_footus.drop(columns = 'to_delete')

In [8]:
df

,Date,team,players,localisation
0,2015-01-02,Bengals,James Wright (c),knee injury (out)
1,2015-01-02,Cardinals,Drew Stanton,knee injury (out)
2,2015-01-02,Colts,Hugh Thornton,shoulder injury (out)
3,2015-01-02,Lions,Larry Warford,knee injury (out)
4,2015-01-02,Ravens,Eugene Monroe,ankle injury (out)
...,...,...,...,...
6318,2022-01-21,Buccaneers,Breshad Perriman,"abdomen, hip injuries (out)"
6319,2022-01-21,Buccaneers,Ronald Jones II,ankle injury (out)
6320,2022-01-21,Rams,Andrew Whitworth,knee injury (out)
6321,2022-01-21,Rams,Taylor Rapp,concussion (out)


In [9]:
df["localisation"].value_counts()

knee injury (out)                                                                          1092
ankle injury (out)                                                                          928
concussion (out)                                                                            802
hamstring injury (out)                                                                      762
foot injury (out)                                                                           338
                                                                                           ... 
knee, toe injuries (out)                                                                      1
ankle, groin injuries (out)                                                                   1
Achilles injury (out for season)                                                              1
oblique muscle injury (out)                                                                   1
tool leave of absense as assistant head 

In [11]:
df.localisation.str.contains("knee").value_counts()

False    5176
True     1147
Name: localisation, dtype: int64

In [14]:
df["team"].value_counts()

Colts         306
Giants        263
Saints        256
49ers         236
Eagles        232
Titans        228
Buccaneers    227
Texans        218
Bengals       217
Steelers      215
Vikings       215
Cowboys       213
Browns        202
Lions         199
Falcons       198
Bills         195
Broncos       195
Seahawks      194
Jets          191
Panthers      191
Chiefs        188
Chargers      185
Packers       182
Jaguars       181
Cardinals     179
Redskins      175
Raiders       162
Ravens        151
Bears         131
Patriots      129
Dolphins      117
Rams           91
Washington     61
Name: team, dtype: int64

In [15]:
new = df.groupby(["team","Date"])
print(new)

In [16]:
df.to_csv("injury in NFL.csv", sep = ";")